In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
# Đọc dữ liệu từ file training
with open('../data/raw/aug_train.csv', 'r', encoding='utf-8') as f:
    header = f.readline().strip().split(',')
data = np.genfromtxt('../data/raw/aug_train.csv',delimiter = ',', dtype =str, skip_header = 1, encoding = 'utf-8')
print(header)
print(data[:1,:])

['enrollee_id', 'city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target']
[['8949' 'city_103' '0.92' 'Male' 'Has relevent experience'
  'no_enrollment' 'Graduate' 'STEM' '>20' '' '' '1' '36' '1']]


## Thông tin các feature trong data
- enrollee_id            : ID ứng viên
- city                   : mã thành phố
- city_development_index : Chỉ số phát triển của thành phố (0-1)
- gender                 : giới tính
- relevent_experience    : Ứng viên có kinh nghiệm liên quan hay không
- enrolled_university    : Loại chương trình đại học đang theo học (nếu có)
- education_level        : Trình độ học vấn
- major_discipline       : Chuyên ngành đại học
- experience             : Tổng số năm kinh nghiệm
- company_size           : Quy mô công ty hiện tại
- company_type           : Loại công ty hiện tại
- last_new_job           : Thời gian giữa hai công việc gần nhất
- training_hours         : Tổng số giờ training đã hoàn thành
- target                 : muốn đổi việc hay không (0/1)

# TIỀN XỬ LÝ DỮ LIỆU
Các câu hỏi được đưa ra
- Dữ liệu có bao nhiêu hàng, bao nhiêu cột
- Kiểu dữ liệu của từ cột đã hợp lý hay chưa
- Gía trị của các ô từng cột đã hợp lý chưa
- Nếu là dữ liệu số thì mean, median, max, min của cột đó là gì
- Có cần bỏ các giá trị ngoại lai đó không, hay là thay bằng mean hoặc median
- Kiểm tra các cột (không phải có giá trị số), có giá trị nào bất thường không => có nên loại bỏ hay thay thế thành giá trị khác
- Cột nào cần chuẩn hóa dữ liệu
- Có dữ liệu nào bị thiếu không, có ảnh hưởng gì không
- Cần phải thêm giá trị gì vào những ô thiếu đó (unknown hay 0,..)

## Kiểm tra tính hợp lệ của dữ liệu

In [3]:
# Dữ liệu có bao nhiêu dòng, cột
n_row, n_col = data.shape
print(f"Tổng số dòng và cột của bộ dữ liệu là: {n_row} dòng và {n_col} cột")

Tổng số dòng và cột của bộ dữ liệu là: 19158 dòng và 14 cột


In [4]:
# Kiểm tra kiểu dữ liệu của từng cột
for i in range(len(header)):
    print (type(data[0,i]))

<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.str_'>


- Hiện tại toàn bộ kiểu dữ liệu của data đang là kiểu str
- Trong khi đó city_development_index,training_hours,target là kiểu số
- experience và last_new_job đặc biệt hơn vì có các phép so sánh ">" hay "<"
=> chuyển các cột trên thành dạng số
- Các cột còn lại kiểu str là hợp lý

In [5]:
# Xây dựng hàm chuyển đổi số nguyên
def safe_int(x):
    try:
        return int(float(x))
    except:
        return np.nan

In [6]:
# Xây dựng hàm chuyển đổi số thực
def safe_float(x):
    try:
        return float(x)
    except:
        return np.nan


In [7]:
unique_exp = np.unique(data[:, 8])   
print(" Các giá trị khác nhau của cột experience:\n", unique_exp)

 Các giá trị khác nhau của cột experience:
 ['' '1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '3' '4'
 '5' '6' '7' '8' '9' '<1' '>20']


- Các giá trị nhận được trong cột experience là các số, '<1','>20' và có tồn tại ô trống
- Xây dựng hàm chuyển đổi từ string sang số cho cột experience, tuân theo quy ước sau
  * Các ô trống sẽ bỏ qua
  * Các số sẽ chuyển đổi bình thường
  * '<1' sẽ quy về thành 0
  * '>20' sẽ quy thành 21

In [8]:
# Xây dựng hàm chuyển đổi cho cột experience
def parse_experience(x):
    if x == "<1":
        return 0
    if x == ">20":
        return 21
    try:
        return int(x)
    except:
        return np.nan

In [9]:
unique_exp = np.unique(data[:, 11])   
print(" Các giá trị khác nhau của cột last_new_job:\n", unique_exp)

 Các giá trị khác nhau của cột last_new_job:
 ['' '1' '2' '3' '4' '>4' 'never']


- Các giá trị nhận được trong cột last_new_job là các số, 'never','>20' và có tồn tại ô trống
- Xây dựng hàm chuyển đổi từ string sang số cho cột last_new_job, tuân theo quy ước sau
  * Các ô trống sẽ bỏ qua
  * Các số sẽ chuyển đổi bình thường
  * 'never' sẽ quy về thành 0
  * '>4' sẽ quy thành 5

In [10]:
def parse_last_new_job(x):
    if x == "never":
        return 0
    if x == ">4":
        return 5
    try:
        return int(x)
    except:
        return np.nan

In [11]:
city_dev = np.array([safe_float(x) for x in data[:, 2]])
training_hours = np.array([safe_int(x) for x in data[:, 12]])
targets = np.array([safe_int(x) for x in data[:, 13]])
experience = np.array([parse_experience(x) for x in data[:, 8]])
last_new_job = np.array([parse_last_new_job(x) for x in data[:,11]])

## Kiểm tra giá trị từng cột đã hợp lý chưa

In [12]:
for i in range((len(header))):
    unique_exp = np.unique(data[:, i])
    print(f"Các giá trị khác nhau của cột {header[i]} là:\n {unique_exp} \n")

Các giá trị khác nhau của cột enrollee_id là:
 ['1' '10' '10000' ... '9992' '9995' '9998'] 

Các giá trị khác nhau của cột city là:
 ['city_1' 'city_10' 'city_100' 'city_101' 'city_102' 'city_103' 'city_104'
 'city_105' 'city_106' 'city_107' 'city_109' 'city_11' 'city_111'
 'city_114' 'city_115' 'city_116' 'city_117' 'city_118' 'city_12'
 'city_120' 'city_121' 'city_123' 'city_126' 'city_127' 'city_128'
 'city_129' 'city_13' 'city_131' 'city_133' 'city_134' 'city_136'
 'city_138' 'city_139' 'city_14' 'city_140' 'city_141' 'city_142'
 'city_143' 'city_144' 'city_145' 'city_146' 'city_149' 'city_150'
 'city_152' 'city_155' 'city_157' 'city_158' 'city_159' 'city_16'
 'city_160' 'city_162' 'city_165' 'city_166' 'city_167' 'city_171'
 'city_173' 'city_175' 'city_176' 'city_179' 'city_18' 'city_180'
 'city_19' 'city_2' 'city_20' 'city_21' 'city_23' 'city_24' 'city_25'
 'city_26' 'city_27' 'city_28' 'city_30' 'city_31' 'city_33' 'city_36'
 'city_37' 'city_39' 'city_40' 'city_41' 'city_42' 'ci

In [13]:
# Đối với dữ liệu số (city_development_index, training_hours, experience,last_new_jobe)
mean_city_dev = np.nanmean(city_dev)
median_city_dev = np.nanmedian(city_dev)
max_city_dev = np.nanmax(city_dev)
min_city_dev = np.nanmin(city_dev)
print("City development index: ")
print(f" Mean: {mean_city_dev}, median: {median_city_dev}, max: {max_city_dev}, min: {min_city_dev}")

mean_train_hour = np.nanmean(training_hours)
median_train_hour = np.nanmedian(training_hours)
max_train_hour = np.nanmax(training_hours)
min_train_hour = np.nanmin(training_hours)
print("Training hour: ")
print(f" Mean: {mean_train_hour}, median: {median_train_hour}, max: {max_train_hour}, min: {min_train_hour}")

mean_experience = np.nanmean(experience)
median_experience = np.nanmedian(experience)
max_experience = np.nanmax(experience)
min_experience = np.nanmin(experience)
print("Experience: ")
print(f" Mean: {mean_experience}, median: {median_experience}, max: {max_experience}, min: {min_experience}")

mean_last_new_job = np.nanmean(last_new_job)
median_last_new_job = np.nanmedian(last_new_job)
max_last_new_job = np.nanmax(last_new_job)
min_last_new_job = np.nanmin(last_new_job)
print("Last new job: ")
print(f" Mean: {mean_last_new_job}, median: {median_last_new_job}, max: {max_last_new_job}, min: {min_last_new_job}")

City development index: 
 Mean: 0.8288480008351602, median: 0.903, max: 0.9490000000000001, min: 0.44799999999999995
Training hour: 
 Mean: 65.36689633573442, median: 47.0, max: 336, min: 1
Experience: 
 Mean: 10.100141413083328, median: 9.0, max: 21.0, min: 0.0
Last new job: 
 Mean: 2.0004270082732853, median: 1.0, max: 5.0, min: 0.0


- Nhìn chung các chỉ số của city_development_index, training_hours, experience,last_new_job không có điểm bất thường

In [14]:
# Kiểm tra số lượng dữ liệu của từng category
cat_cols = [
    'gender',
    'relevent_experience',
    'enrolled_university',
    'education_level',
    'major_discipline',
    'company_size',
    'company_type',
    'target'
]
cat_cols = [c for c in cat_cols if c in header]
print(cat_cols)
for c in cat_cols:
    idx = header.index(c)
    col = data[:, idx]
    # Các ô rỗng được mặc định là missing
    col_filled = np.array([x if x != '' else 'Missing' for x in col])
    unique_vals, counts = np.unique(col_filled, return_counts=True)
    total = counts.sum()
    print(f"Column: {c}")
    print(f"  Unique values: {len(unique_vals)}")
    for val, cnt in zip(unique_vals, counts):
        pct = cnt / total * 100
        print(f"    {val}: {cnt} ({pct:.2f}%)")
    print()


['gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'company_size', 'company_type', 'target']
Column: gender
  Unique values: 4
    Female: 1238 (6.46%)
    Male: 13221 (69.01%)
    Missing: 4508 (23.53%)
    Other: 191 (1.00%)

Column: relevent_experience
  Unique values: 2
    Has relevent experience: 13792 (71.99%)
    No relevent experience: 5366 (28.01%)

Column: enrolled_university
  Unique values: 4
    Full time course: 3757 (19.61%)
    Missing: 386 (2.01%)
    Part time course: 1198 (6.25%)
    no_enrollment: 13817 (72.12%)

Column: education_level
  Unique values: 6
    Graduate: 11598 (60.54%)
    High School: 2017 (10.53%)
    Masters: 4361 (22.76%)
    Missing: 460 (2.40%)
    Phd: 414 (2.16%)
    Primary School: 308 (1.61%)

Column: major_discipline
  Unique values: 7
    Arts: 253 (1.32%)
    Business Degree: 327 (1.71%)
    Humanities: 669 (3.49%)
    Missing: 2813 (14.68%)
    No Major: 223 (1.16%)
    Other: 381 (1.99%)
   

- Bộ dữ liệu training này có phần trăm nam ứng viên rất cao (69.01%) cho thấy sự bias trong bộ dữ liệu, vì vậy sẽ không xét tới yếu tố giới tính trong quá trình khám phá dữ liệu vì có khả năng cao đưa ra kết luận không đúng
- Ngoài ra, yếu tố company_size cũng sẽ không xét trong quá trình khám phá vì chưa đánh gia sđược thang đo của yếu tố để đưa ra kết luận

### Kiểm tra missing value

In [15]:
# Số lượng ô bị thiếu cho từng cột
missing_counts = np.sum(data == '' , axis=0)
print(header)
missing_counts

['enrollee_id', 'city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target']


array([   0,    0,    0, 4508,    0,  386,  460, 2813,   65, 5938, 6140,
        423,    0,    0])

### Điền các giá trị vào ô còn thiếu

In [16]:
def fill_missing_with_median(arr):
    clean = arr[~np.isnan(arr)]
    median = np.median(clean)
    arr[np.isnan(arr)] = median
    return arr

city_dev = fill_missing_with_median(city_dev)
training_hours = fill_missing_with_median(training_hours)
experience = fill_missing_with_median(experience)
last_new_job = fill_missing_with_median(last_new_job)


In [17]:
def label_encode(col):
    unique_vals = np.unique(col)
    mapping = {val: i for i, val in enumerate(unique_vals)}
    encoded = np.array([mapping[x] for x in col])
    return encoded, mapping

city_encoded, city_map = label_encode(data[:, 1])
company_type_encoded, comp_type_map = label_encode(data[:, 10])


In [18]:
company_size_map = {
    "<10": 1,
    "10/49": 2,
    "50-99": 3,
    "100-500": 4,
    "500-999": 5,
    "1000-4999": 6,
    "5000-9999": 7,
    "10000+": 8
}

def encode_company_size(x):
    return company_size_map.get(x, np.nan)

company_size = np.array([encode_company_size(x) for x in data[:, 9]], dtype=float)
company_size = fill_missing_with_median(company_size)


In [19]:
def min_max_norm(arr):
    mn = np.min(arr)
    mx = np.max(arr)
    return (arr - mn) / (mx - mn + 1e-9)

city_dev_norm = min_max_norm(city_dev)
training_hours_norm = min_max_norm(training_hours)
experience_norm = min_max_norm(experience)
last_new_job_norm = min_max_norm(last_new_job)


In [20]:
def zscore(arr):
    return (arr - np.mean(arr)) / (np.std(arr) + 1e-9)

city_dev_z = zscore(city_dev)
experience_z = zscore(experience)
target = data[:, -1].astype(float)

In [22]:
final_data_with_target = np.column_stack([
    city_dev_norm,
    training_hours_norm,
    experience_norm,
    last_new_job_norm,
    company_type_encoded,
    company_size,
    target  
])

# Header mới
header = "city_dev_norm,training_hours_norm,experience_norm,last_new_job_norm,company_type_encoded,company_size,target"

# Lưu CSV
np.savetxt("../data/processed/processed_data.csv", final_data_with_target, delimiter=",", header=header, comments='', fmt='%f')
